In [49]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import math
import imutils
import glob
from PIL import Image

In [ ]:
# write kalman filter for tracking points on image


In [3]:
drone_img_path = "test_data/drone/00042.jpeg"
satellite_img_path = "test_data/satellite/1.png"

In [4]:
drone_img = cv2.imread(drone_img_path)
drone_gray = cv2.cvtColor(drone_img, cv2.COLOR_BGR2GRAY)

satellite_img = cv2.imread(satellite_img_path)
satellite_gray = cv2.cvtColor(satellite_img, cv2.COLOR_BGR2GRAY)

In [12]:
def get_template(img_path):
    template = cv2.imread(img_path)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.resize(template, (640,480))
    template = cv2.Canny(template, 100, 300)
    return template

In [46]:
i, j = 0, 1
for imagePath in sorted(glob.glob("tiles-20-03/jpeg/*.jpeg")):
	# loop over the images to 
	image = cv2.imread(f"tiles-20-03/Tiles/{j}.png")
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	# load the image, convert it to grayscale, and initialize the
	# bookkeeping variable to keep track of the matched region
	found = None
	# loop over the scales of the image
	template = get_template(imagePath)
	(tH, tW) = template.shape[:2]
	for scale in np.linspace(0.1, 1.0, 10)[::-1]:
		# resize the image according to the scale, and keep track
		# of the ratio of the resizing
		resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
		r = gray.shape[1] / float(resized.shape[1])
		# if the resized image is smaller than the template, then break
		# from the loop
		if resized.shape[0] < tH or resized.shape[1] < tW:
			break
		# detect edges in the resized, grayscale image and apply template
		# matching to find the template in the image
		edged = cv2.Canny(resized, 50, 200)
		result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
		(_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
		# check to see if the iteration should be visualized
		if False:
			# draw a bounding box around the detected region
			print((maxLoc[0], maxLoc[1]))
			clone = np.dstack([edged, edged, edged])
			cv2.rectangle(clone, (maxLoc[0], maxLoc[1]),
				(maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
			cv2.imshow("Visualize", clone)
		# if we have found a new maximum co	rrelation value, then update
		# the bookkeeping variable
		if found is None or maxVal > found[0]:
			found = (maxVal, maxLoc, r)
	# unpack the bookkeeping variable and compute the (x, y) coordinates
	# of the bounding box based on the resized ratio
	(_, maxLoc, r) = found
	(startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
	(endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
	# draw a bounding box around the detected result and display the image
	cv2.circle(image, ((startX + endX) // 2,(startY + endY) //2), 5, (0, 255, 0), 2)
	# print(imagePath)
	cv2.imwrite(imagePath.replace("tiles-20-03/jpeg/", "results/template-matching2/"), image)
	i += 1
	if i % 12 == 0:
		j += 2
		print(i)

12
24
36
48
60
72
84
96
108
120
132
144
156
168
180
192
204
216


[ WARN:0@12897.366] global loadsave.cpp:244 findDecoder imread_('tiles-20-03/Tiles/37.png'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [47]:
def stitch_images(drone_path, rpath, outpath):
    image1 = Image.open(drone_path)
    image1 = image1.resize((2096,1395))
    image2 = Image.open(rpath)

    # Determine the size of the output image
    width, height = image1.size
    width += image2.size[0]

    # Create a new blank image to hold the stitched images
    output_image = Image.new("RGB", (width, height))

    # Paste the input images onto the output image
    output_image.paste(image1, (0, 0))
    output_image.paste(image2, (image1.size[0], 0))

    # Save the output image
    output_image.save(outpath)

In [48]:
drone_results_imgs = zip(sorted(glob.glob("tiles-20-03/jpeg/*.jpeg")), sorted(glob.glob("results/template-matching2/*.jpeg")))
for x,y in drone_results_imgs:
    stitch_images(x,y, x.replace("tiles-20-03/jpeg/", "results/tm-stitched/"))

In [22]:
drone_results_imgs